In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
label2id = {'negative':0,'positive':1}

train = pd.read_csv(r'train.csv', encoding='utf_8_sig')
train = train.replace({"label": label2id})
train.fillna('', inplace=True)
test = pd.read_csv(r'test.csv', encoding='utf_8_sig')
test = test.replace({"label": label2id})
test.fillna('', inplace=True)

/tmp/ipykernel_1985/2093326092.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train = train.replace({"label": label2id})
/tmp/ipykernel_1985/2093326092.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test = test.replace({"label": label2id})


In [4]:
train

,label,text,diff
0,0,Merge branch '1.8.x',diff --git a/src/ClientCommand.cpp b/src/Clien...
1,1,SPOOLSS: Try to avoid an infinite loop.\n\nUse...,diff --git a/epan/dissectors/packet-dcerpc-spo...
2,1,CVE-2017-12895/ICMP: Check the availability of...,diff --git a/print-icmp.c b/print-icmp.c\ninde...
3,0,Change distribution URL.\n\ngit-svn-id: https:...,diff --git a/c/xml-security-c.spec b/c/xml-sec...
4,1,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/png.c b/coders/png.c\ninde...
5,0,2021 license,diff --git a/Gruntfile.js b/Gruntfile.js\ninde...
6,0,Merge pull request #2426 from alvarobartt/deve...,diff --git a/nltk/tag/stanford.py b/nltk/tag/s...
7,0,Fixed link errors,diff --git a/src/irisnet/CMakeLists.txt b/src/...
8,1,[MJ2] Avoid index out of bounds access to pi->...,diff --git a/src/lib/openmj2/pi.c b/src/lib/op...
9,0,Bootstrap 3 migration: alert-error->alert-danger.,diff --git a/src/main/java/org/projectforge/we...


In [5]:
len(train)

20

In [6]:
train['label'].value_counts()

label
0    13
1     7
Name: count, dtype: int64

In [7]:
test['label'].value_counts()

label
0    6334
1    3758
Name: count, dtype: int64

In [8]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
# model_id = "../../sentence-transformers/paraphrase-mpnet-base-v2"
model = SentenceTransformer('../../sentence-transformers/paraphrase-mpnet-base-v2')

In [9]:
train['embeddings'] = train['text'].apply(lambda x: model.encode(str(x)))
test['embeddings'] = test['text'].apply(lambda x: model.encode(str(x)))

In [10]:
test

,label,text,diff,embeddings
0,0,Header: clarify that newline conversion is raw...,diff --git a/lib/mail/header.rb b/lib/mail/hea...,"[-0.13944657, 0.07307367, 0.04000201, -0.04878..."
1,0,[ticket/16691] Fix illegal offset for report_p...,diff --git a/phpBB/phpbb/notification/type/rep...,"[-0.14759511, 0.09322985, 0.033147767, 0.06086..."
2,0,Hide the Ulterius Agent when its ran.,diff --git a/UlteriusAgent/Program.cs b/Ulteri...,"[0.0017122248, 0.030022504, -0.0066839927, 0.2..."
3,0,Move DOM lookup outside of loop\n\nfixes #1360,diff --git a/lib/ext/ui.js b/lib/ext/ui.js\nin...,"[-0.042559765, 0.13228251, -0.09083305, 0.1190..."
4,1,remove /login from tracking logs so we don't c...,diff --git a/auth/views.py b/auth/views.py\nin...,"[-0.052938223, 0.12112431, 0.077534795, 0.1962..."
...,...,...,...,...
10087,1,disallow boot from volume from specifying arbi...,diff --git a/nova/compute/api.py b/nova/comput...,"[-0.041516595, -0.017604265, 0.0018733353, 0.0..."
10088,1,writet1 protection against buffer overflow\n\n...,diff --git a/texk/dvipsk/ChangeLog b/texk/dvip...,"[-0.15439494, 0.04666118, 0.046322543, 0.12190..."
10089,1,PR/398: Correctly truncate pascal strings (fix...,diff --git a/src/softmagic.c b/src/softmagic.c...,"[0.026086327, -0.08980047, -0.029941613, 0.127..."
10090,0,Add paragraph on GitHub deprecating passwords ...,diff --git a/CONTRIBUTING-TO-ESAPI.txt b/CONTR...,"[0.10408172, 0.08221948, 0.02908223, 0.0035576..."


In [11]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [12]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [13]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Precision: 0.7062131682518205
Logistic Regression Recall: 0.711850971066191
Logistic Regression Recall: 0.6930508408065337
Logistic Regression Accuracy: 0.711850971066191
